In [ ]:
import pandas as pd

In [ ]:
temperature_data = pd.read_csv('/content/temperature.csv')

In [ ]:
co2_data = pd.read_csv('/content/carbon_emmission.csv')

**İki veri seti var:**

**temperature_data:** yıllarca santigrat derece cinsinden ölçülen yıllık sıcaklık anomalileri.

**co2_data:** Milyonda parça (ppm) cinsinden aylık küresel atmosferik CO₂ konsantrasyonları.

In [ ]:
temperature_data_preview = temperature_data.head()
co2_data_preview = co2_data.head()

temperature_data_preview, co2_data_preview

(   ObjectId                       Country ISO2 ISO3  F1961  F1962  F1963  \
 0         1  Afghanistan, Islamic Rep. of   AF  AFG -0.113 -0.164  0.847   
 1         2                       Albania   AL  ALB  0.627  0.326  0.075   
 2         3                       Algeria   DZ  DZA  0.164  0.114  0.077   
 3         4                American Samoa   AS  ASM  0.079 -0.042  0.169   
 4         5      Andorra, Principality of   AD  AND  0.736  0.112 -0.752   
 
    F1964  F1965  F1966  ...  F2013  F2014  F2015  F2016  F2017  F2018  F2019  \
 0 -0.764 -0.244  0.226  ...  1.281  0.456  1.093  1.555  1.540  1.544  0.910   
 1 -0.166 -0.388  0.559  ...  1.333  1.198  1.569  1.464  1.121  2.028  1.675   
 2  0.250 -0.100  0.433  ...  1.192  1.690  1.121  1.757  1.512  1.210  1.115   
 3 -0.140 -0.562  0.181  ...  1.257  1.170  1.009  1.539  1.435  1.189  1.539   
 4  0.308 -0.490  0.415  ...  0.831  1.946  1.690  1.990  1.925  1.919  1.964   
 
    F2020  F2021  F2022  
 0  0.498  1.327  2.01

sıcaklık ve karbondioksit verilerinin istatistiksel **analizi**

In [ ]:
temperature_values = temperature_data.filter(regex='^F').stack()
temperature_stats = {
    "Mean": temperature_values.mean(),
    "Median": temperature_values.median(),
    "Variance": temperature_values.var()
}


co2_values = co2_data["Value"]
co2_stats = {
    "Mean": co2_values.mean(),
    "Median": co2_values.median(),
    "Variance": co2_values.var()
}

temperature_stats, co2_stats

({'Mean': np.float64(0.5377713483146068),
  'Median': 0.47,
  'Variance': 0.4294524831504378},
 {'Mean': np.float64(180.71615286624203),
  'Median': 313.835,
  'Variance': 32600.00200469294})

Ortalama sıcaklık değişimi yaklaşık 0,54°C'dir, medyan 0,47°C ve varyans 0,43'tür ve bu da sıcaklık anomalilerinde hafif değişkenlik olduğunu gösterir. CO₂ konsantrasyonları için ortalama 180,72 ppm'dir, medyan 313,84 ppm'de önemli ölçüde daha yüksektir ve varyans 32.600'dür ve bu da veri setinin zaman dilimi boyunca CO₂ seviyelerinde önemli değişkenliği yansıtır. Bu, CO₂ verilerindeki sıcaklık değişikliklerine kıyasla daha güçlü dalgalanmayı vurgular.

# **ZAMAN SERİSİ ANALİZİ**

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
temperature_years = temperature_data.filter(regex='^F').mean(axis=0)
temperature_years.index = temperature_years.index.str.replace('F', '').astype(int)


In [ ]:
co2_data['Year'] = co2_data['Date'].str[:4].astype(int)
co2_yearly = co2_data.groupby('Year')['Value'].mean()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=temperature_years.index, y=temperature_years.values,
    mode='lines+markers', name="Sıcaklık Değişimi (°C)"
))
fig.add_trace(go.Scatter(
    x=co2_yearly.index, y=co2_yearly.values,
    mode='lines+markers', name="CO₂ Konsantrasyonu (ppm)", line=dict(dash='dash')
))
fig.update_layout(
    title="Sıcaklık Değişimi ve CO₂ Konsantrasyonlarının Zaman Serisi",
    xaxis_title="YILLAR",
    yaxis_title="Degerler",
    template="plotly_white",
    legend_title="Metrikler"
)
fig.show()


In [ ]:
merged_data = pd.DataFrame({
    "Sıcaklık değişimi": temperature_years,
    "CO₂ Konsantrasyonu": co2_yearly
}).dropna()

heatmap_fig = px.imshow(
    merged_data.corr(),
    text_auto=".2f",
    color_continuous_scale="RdBu",
    title="Korelasyon Isı Haritası"
)
heatmap_fig.update_layout(
    template="plotly_white"
)
heatmap_fig.show()

In [ ]:
scatter_fig = px.scatter(
    merged_data,
    x="CO₂ Konsantrasyonu",
    y="Sıcaklık değişimi",
    labels={"CO₂ Konsantrasyonu": "CO₂ Concentration (ppm)", "Sıcaklık değişimi": "Temperature Change (°C)"},
    title="Temperature Change vs CO₂ Concentration",
    template="plotly_white"
)
scatter_fig.update_traces(marker=dict(size=10, opacity=0.7))
scatter_fig.show()

# **iki değişkenin arasındaki doğrusal ilşkiyi analz edeceğm **
slope: eğim

intercept: y-kesişim

r_value, p_value, std_err: korelasyon ve istatistiksel bilgiler
go ; etkileşimli grafikler

In [ ]:
from scipy.stats import linregress

In [ ]:
temp_trend = linregress(temperature_years.index, temperature_years.values)
temp_trend_line = temp_trend.slope

In [ ]:
co2_trend = linregress(co2_yearly.index, co2_yearly.values)
co2_trend_line = co2_trend.slope
fig_trends = go.Figure()


In [ ]:

temp_trend = linregress(temperature_years.index, temperature_years.values)
temp_trend_line = temp_trend.slope * temperature_years.index + temp_trend.intercept


co2_trend = linregress(co2_yearly.index, co2_yearly.values)
co2_trend_line = co2_trend.slope * co2_yearly.index + co2_trend.intercept

fig_trends = go.Figure()


fig_trends.add_trace(go.Scatter(
    x=temperature_years.index,
    y=temperature_years.values,
    mode='lines+markers',
    name="Sıcaklık Değişimi (°C)"
))

fig_trends.add_trace(go.Scatter(
    x=temperature_years.index,
    y=temp_trend_line,
    mode='lines',
    name=f"Sıcaklık Eğilimi (Eğim: {temp_trend.slope:.2f})",
    line=dict(dash='dash')
)
)

fig_trends.add_trace(go.Scatter(
    x=co2_yearly.index,
    y=co2_yearly.values,
    mode='lines+markers',
    name="CO₂ Konsantrasyonu (ppm)"
))

fig_trends.add_trace(go.Scatter(
    x=co2_yearly.index,
    y=co2_trend_line,
    mode='lines',
    name=f"CO₂ Eğilimi (Eğim: {co2_trend.slope:.2f})",
    line=dict(dash='dash')
)
)


fig_trends.update_layout(
    title="Sıcaklık Değişimi ve CO₂ Konsantrasyonundaki Eğilimler",
    xaxis_title="Yıl",
    yaxis_title="Değerler",
    template="plotly_white",
    legend_title="Metrikler"
)

fig_trends.show()

In [ ]:

co2_data['Month'] = co2_data['Date'].str[-2:].astype(int)
co2_monthly = co2_data.groupby('Month')['Value'].mean()

fig_seasonal = px.line(
    co2_monthly,
    x=co2_monthly.index,
    y=co2_monthly.values,
    labels={"x": "Ay", "y": "CO₂ Konsantrasyonu (ppm)"},
    title="CO₂ Konsantrasyonundaki Mevsimsel Değişimler",
    markers=True
)

fig_seasonal.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=list(range(1, 13))
    ),
    template="plotly_white"
)

fig_seasonal.show()

# **KORELASYON VE NEDENSELLİK ANALİZİ**

pearsonr: Doğrusal ilişki var mı?

spearmanr: Sıralama ilişkisi var mı?

grangercausalitytests: CO₂, sıcaklığı  önceden tahmin edebilir mi?

In [ ]:
from scipy.stats import pearsonr, spearmanr
from statsmodels.tsa.stattools import grangercausalitytests

In [ ]:

pearson_corr  = pearsonr(merged_data["CO₂ Konsantrasyonu"], merged_data["Sıcaklık değişimi"])
spearman_corr, _ = spearmanr(merged_data["CO₂ Konsantrasyonu"], merged_data["Sıcaklık değişimi"])


granger_data = merged_data.diff().dropna()


granger_input = granger_data[["Sıcaklık değişimi", "CO₂ Konsantrasyonu"]]

granger_results = grangercausalitytests(granger_input, maxlag=3, verbose=False)

granger_p_values = {f"Gecikme {lag}": round(results[0]['ssr_chi2test'][1], 4)
                    for lag, results in granger_results.items()}


pearson_corr, spearman_corr, granger_p_values

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning:

verbose is deprecated since functions should not print results



(PearsonRResult(statistic=np.float64(0.9554282559257312), pvalue=np.float64(1.7347463241961698e-33)),
 np.float64(0.9379013371609882),
 {'Gecikme 1': np.float64(0.017),
  'Gecikme 2': np.float64(0.6562),
  'Gecikme 3': np.float64(0.4064)})

**Gecikmeli Etki Analizi**

Gecikmeli değişken = bir değişkenin geçmişteki değeri

Zaman serilerinde bugünkü olaylar, genellikle geçmişteki gelişmelere bağlıdır.

In [ ]:
import statsmodels.api as sm


In [ ]:

merged_data['CO₂ Gecikme 1'] = merged_data["CO₂ Konsantrasyonu"].shift(1)
merged_data['CO₂ Gecikme 2'] = merged_data["CO₂ Konsantrasyonu"].shift(2)
merged_data['CO₂ Gecikme 3'] = merged_data["CO₂ Konsantrasyonu"].shift(3)


lagged_data = merged_data.dropna()


X = lagged_data[['CO₂ Konsantrasyonu', 'CO₂ Gecikme 1', 'CO₂ Gecikme 2', 'CO₂ Gecikme 3']]  # İsim çevirisi

y = lagged_data['Sıcaklık değişimi']
X = sm.add_constant(X)


model = sm.OLS(y, X).fit()


model_summary = model.summary()
model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Sıcaklık değişimi   R-squared:                       0.949
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     252.5
Date:                Mon, 12 May 2025   Prob (F-statistic):           2.97e-34
Time:                        10:09:24   Log-Likelihood:                 45.098
No. Observations:                  59   AIC:                            -80.20
Df Residuals:                      54   BIC:                            -69.81
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -4.7980      0.317    -15.137      0.000      -5.434      -4.163
CO₂ Konsantrasyonu     0.3245      0.055      5.942      0.000       0.215       0.434
CO₂ Gecikme 1         -0.2962      0.068     -4.361      0.000      -0.432      -0.160
CO₂ Gecikme 2          0.0104      0.068      0.153      0.879      -0.126       0.146
CO₂ Gecikme 3         -0.0107      0.056     -0.191      0.849      -0.123       0.101
==============================================================================
Omnibus:                        2.369   Durbin-Watson:                   1.554
Prob(Omnibus):                  0.306   Jarque-Bera (JB):                2.077
Skew:                          -0.457   Prob(JB):                        0.354
Kurtosis:                       2.902   Cond. No.                     7.54e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.54e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

CO₂ seviyesi sıcaklık değişimini açıkça etkiliyor.

Ancak uzun vadeli gecikmelerin (2 ve 3 yıl) etkisi gözlemlenemiyor.

Bu, ısınma etkisinin daha kısa vadeli CO₂ artışlarıyla ilişkili olabileceğini düşündürüyor.

## **Kümeleme İklim Desenleri**

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:

clustering_data = merged_data[["Sıcaklık değişimi", "CO₂ Konsantrasyonu"]].dropna()

scaler = StandardScaler()
scaled_data = scaler.fit_transform(clustering_data)


kmeans = KMeans(n_clusters=3, random_state=42)
clustering_data['Küme'] = kmeans.fit_predict(scaled_data)


clustering_data['Etiket'] = clustering_data['Küme'].map({
    0: 'Orta Seviye Sıcaklık ve CO₂',
    1: 'Yüksek Sıcaklık ve CO₂',
    2: 'Düşük Sıcaklık ve CO₂'
})

fig_clusters = px.scatter(
    clustering_data,
    x="CO₂ Konsantrasyonu",
    y="Sıcaklık değişimi",
    color="Etiket",
    color_discrete_sequence=px.colors.qualitative.Set2,
    labels={
        "CO₂ Konsantrasyonu": "CO₂ Konsantrasyonu (ppm)",
        "Sıcaklık değişimi": "Sıcaklık Değişimi (°C)",
        "Etiket": "İklim Örüntüsü"
    },
    title="İklim Örüntülerine Göre Yılların Kümelemesi"
)

fig_clusters.update_layout(
    template="plotly_white",
    legend_title="İklim Örüntüsü"
)

fig_clusters.show()

**GRAFİK ANALİZİ**

 -CO₂ konsantrasyonu arttıkça, sıcaklık değişimi de artıyor.

 -Bu durum, küresel ısınmanın CO₂ artışına bağlı olduğunu doğrudan görsel olarak destekliyor.

 -Noktalar soldan sağa (yeşilden maviye) geçerken düzenli bir artış gözleniyor

 -Bu drumda zaman içinde CO₂ birikimiyle sıcaklığın da kademeli olarak arttığını gösteriyor

 -Yıllar sadece CO₂ ve sıcaklık değerlerine göre sınıflandırıldığında bile anlamlı kümeler oluşturabiliyor

 -Bu da iklim değişikliğini açıklamak ve geleceğe yönelik senaryo üretmek için bu iki değişkenin güçlü göstergeler olduğunu gösteriyor



# **Karbon salınımlarının doğrudan sıcaklıkla nasıl ilişkili olduğu matematiksel olarak gösterilmesi**

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression

X = merged_data[["CO₂ Konsantrasyonu"]].values
y = merged_data["Sıcaklık değişimi"].values

model = LinearRegression()
model.fit(X, y)

def simulate_temperature_change(co2_percentage_change):

    current_mean_co2 = merged_data["CO₂ Konsantrasyonu"].mean()
    new_co2 = current_mean_co2 * (1 + co2_percentage_change / 100)

    predicted_temp = model.predict([[new_co2]])
    return predicted_temp[0]

scenarios = {
    "Increase CO₂ by 10%": simulate_temperature_change(10),
    "Decrease CO₂ by 10%": simulate_temperature_change(-10),
    "Increase CO₂ by 20%": simulate_temperature_change(20),
    "Decrease CO₂ by 20%": simulate_temperature_change(-20),
}
scenarios

{'Increase CO₂ by 10%': np.float64(1.0866445037958163),
 'Decrease CO₂ by 10%': np.float64(-0.059993041237237144),
 'Increase CO₂ by 20%': np.float64(1.6599632763123422),
 'Decrease CO₂ by 20%': np.float64(-0.6333118137537621)}


CO₂ %10 artarsa	+1.09°C	CO₂ salımı artarsa sıcaklık belirgin şekilde artıyor.

CO₂ %10 azalırsa	-0.06°C	Küçük bir azalma sıcaklıkta hafif bir düşüşe yol açıyor.

CO₂ %20 artarsa	+1.66°C	Daha büyük bir artış, sıcaklıkta ciddi bir artış demek.

CO₂ %20 azalırsa	-0.63°C	CO₂ azaltımı sıcaklığı önemli ölçüde düşürebiliyor.
